# Mathematical Optimization: Linear and Integer Linear Programming

Dr. Leo Singer<br>
NASA Goddard Space Flight Center<br>
Presented on July 31, 2024 at the [ZTF Summer School 2024](https://www.ztf.caltech.edu/summer-school-2024.html) at the University of Minnesota

Note that the Python packages required for this notebook work on Linux and macOS but not Windows. If you are on Windows, please use the Google Colab notebook linked from the [README file](https://github.com/ZTF-Summer-School/ztf_summer_school_2024/blob/main/lectures/03-milp/README.md).

Before you get started, run this command to install all of the Python packages that you will need...

In [ ]:
!pip install ligo.skymap 'scipy<1.14.0' 'numpy<2' highspy

## Linear programming: standard form

Linear programming is a mathematical optimization technique that is useful for modeling and solving problems related to allocation of limited resources. A linear program is a system of linear equations and inequalities. Any linear program (LP) can be written in the following canonical form:

$$
\begin{align*}
\text{Find the vector } & \mathbf{x} \\
\text{that minimizes } & \mathbf{c}^\mathsf{T}\mathbf{x} \\
\text{subject to the constraints } & \mathbf{A} \mathbf{x} \leq \mathbf{b} \\
\text{and } & \mathbf{x} \geq \mathbf{0}.
\end{align*}
$$

(Note that usually this definition calls for maximization, not minimization, but Scipy's linear programming functions perform minimization.)

## Linear programming example

Prof. Muffinman and Dr. Lemonade need to purchase snacks for day 3 of their summer school. There are 30 students. Each student will write 20 lines of code per cup of tea they drink, 100 per cup of coffee, 100 per muffin, 20 per donut (because the donuts give them tummy aches), and 70 per bagel. Here are the unit prices for the treats:

| Item   | Cost   |
| ------ | ------ |
| Tea    | \$1.00 |
| Coffee | \$2.00 |
| Muffin | \$2.00 |
| Donut  | \$3.50 |
| Bagel  | \$2.00 |

Each student must have at least one drink and at least one pastry, at least one of which must be a donut. The Muffinman-Lemonade lab has a budget of \$180 for snacks. How many of each snack should they buy? How many lines of code do the students write?

**Exercise 1.** Write down a linear cost function and a system of inequalities to model this problem.

**Excercise 2.** Express this problem as LP in canonical form by writing down the values of the array $\mathbf{A}$, the vector $\mathbf{b}$, and the vector $\mathbf{c}$.

**Exercise 3.** Solve the LP problem using [`scipy.optimize.linprog`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html) by completing the code below. How many of each treat should Muffinman and Lemonade order?

In [ ]:
import numpy as np
from scipy import optimize

# Fill in these values
c = ...
A = ...
b = ...

solution = optimize.linprog(c, A, b)

print(f'{-solution.fun:5g} lines of code')
varnames = ['teas', 'coffees', 'muffins', 'donuts', 'bagels']
for count, name in zip(solution.x, varnames):
    print(f'{count:5g} {name}')

## Integer linear programming (ILP), mixed integer linear programming (MILP)

Integer linear programming (ILP) is an extension of LP in which all of the decision variables $\mathbf{x}$ are integers. In mixed integer linear programming (MILP), certain of the decision variables are reals and certain are integers.

**Exercise 4.** The caterer only sells _whole_ cups of coffee and tea and _whole_ pastries. Solve the LP problem using [`scipy.optimize.milp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.milp.html) by completing the code below. How many of each treat should Muffinman and Lemonade order?

In [ ]:
solution = optimize.milp(
    c,
    integrality=np.ones_like(1),  # all variables must be integers
    constraints=optimize.LinearConstraint(A, ub=b)
)

print(f'{-solution.fun:5g} lines of code')
varnames = ['teas', 'coffees', 'muffins', 'donuts', 'bagels']
for count, name in zip(solution.x, varnames):
    print(f'{count:5g} {name}')

**Exercise 5.** There is a sale on bagels: they are half price when you buy them by the dozen. How do you incorporate this into your model?

In [ ]:
import numpy as np
from scipy import optimize

# Fill in these values
c = ...
A = ...
b = ...

solution = optimize.milp(
    c,
    integrality=np.ones_like(c),
    constraints=optimize.LinearConstraint(A, ub=b)
)

print(f'{-solution.fun:5g} lines of code')
varnames = ['teas', 'coffees', 'muffins', 'donuts', 'bagels']
for count, name in zip(solution.x, varnames):
    print(f'{count:5g} {name}')

**Exercise 6.** Model and solve the problem using the [high-level Python interface to the HiGHS solver](https://ergo-code.github.io/HiGHS/dev/interfaces/python/).

This is the same solver that SciPy uses, but it provides a higher-level interface that allows you to input the model symbollically.

In [ ]:
import highspy

model = highspy.Highs()

# Create decision variables
teas = model.addIntegral()
coffees = model.addIntegral()
# fill me in...

# Add constraints
model.addConstr(teas + coffees >= 30)
# fill me in...

# Fill me in
model.maximize(20 * teas + ...)

print(f'{model.getObjectiveValue():5g} lines of code')
varnames = ['teas', 'coffees', 'muffins', 'donuts', 'bagels']
for count, name in zip(model.allVariableValues(), varnames):
    print(f'{count:5g} {name}')

## MILP for astronomical observation planning

Let's use MILP to plan follow-up of a LIGO-Virgo event. We'll use [S231113bw](https://gracedb.ligo.org/superevents/S231113bw/), a BBH merger candidate (originally classified as a NSBH candidate) that ZTF actually followed up ([GCN Circular 35032](https://gcn.nasa.gov/circulars/35032)). We'll download all of the data for you below, so all you have to do is the math! 😉

### Problem setup

We'll use the following Python packages to help us set up the problem:

- [ligo.skymap](https://pypi.org/project/ligo.skymap/) for reading and plotting LIGO-Virgo HEALPix localization maps
- [astropy-healpix](https://pypi.org/project/astropy-healpix/) for converting HEALPix coordinates to RA and Dec
- [healpy](https://pypi.org/project/healpy/) for finding the HEALPix pixels contained within polygon regions
- [astroplan](https://pypi.org/project/astroplan/) for finding when a given ZTF field is observable

First, some imports...

In [ ]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

import astroplan
from astropy.coordinates import ICRS, SkyCoord
from astropy import units as u
from astropy.utils.data import download_file
from astropy.table import QTable
from astropy.time import Time
from astropy_healpix import HEALPix
from ligo.skymap import plot
from ligo.skymap.io import read_sky_map
import healpy as hp
from matplotlib import pyplot as plt
import numpy as np
from tqdm.auto import tqdm

warnings.simplefilter('ignore', astroplan.TargetNeverUpWarning)
warnings.simplefilter('ignore', astroplan.TargetAlwaysUpWarning)

### Read localization map

First, we'll download and plot the sky map.

In [ ]:
skymap, metadata = read_sky_map('https://gracedb.ligo.org/api/superevents/S231113bw/files/bayestar.multiorder.fits')

ax = plt.axes(projection='astro mollweide')
ax.imshow_hpx(skymap, cmap='cylon')
ax.grid()

We can also get the event time from the sky map.

In [ ]:
event_time = Time(metadata['gps_time'], format='gps').utc
event_time.format = 'iso'
event_time

### Model ZTF focal plane footprint

Let's create a model of the footprint of the ZTF focal plane, with the telescope pointed at a reference position of R.A.=0°, Dec.=0°.

In [ ]:
# Table 1 from Bellm et al. (2019)
# http://adsabs.harvard.edu/abs/2019PASP..131a8002B
ns_nchips = 4
ew_nchips = 4
ns_npix = 6144
ew_npix = 6160
plate_scale = 1.01 * u.arcsec
ns_chip_gap = 0.205 * u.deg
ew_chip_gap = 0.140 * u.deg

ns_total = ns_nchips * ns_npix * plate_scale + (ns_nchips - 1) * ns_chip_gap
ew_total = ew_nchips * ew_npix * plate_scale + (ew_nchips - 1) * ew_chip_gap

rcid = np.arange(64)

chipid, rc_in_chip_id = np.divmod(rcid, 4)
ns_chip_index, ew_chip_index = np.divmod(chipid, ew_nchips)
ns_rc_in_chip_index = np.where(rc_in_chip_id <= 1, 1, 0)
ew_rc_in_chip_index = np.where((rc_in_chip_id == 0) | (rc_in_chip_id == 3), 0, 1)

ew_offsets = ew_chip_gap * (ew_chip_index - (ew_nchips - 1) / 2) + ew_npix * plate_scale * (ew_chip_index - ew_nchips / 2) + 0.5 * ew_rc_in_chip_index * plate_scale * ew_npix
ns_offsets = ns_chip_gap * (ns_chip_index - (ns_nchips - 1) / 2) + ns_npix * plate_scale * (ns_chip_index - ns_nchips / 2) + 0.5 * ns_rc_in_chip_index * plate_scale * ns_npix

ew_ccd_corners = 0.5 * plate_scale * np.asarray([ew_npix, 0, 0, ew_npix])
ns_ccd_corners = 0.5 * plate_scale * np.asarray([ns_npix, ns_npix, 0, 0])

ew_vertices = ew_offsets[:, np.newaxis] + ew_ccd_corners[np.newaxis, :]
ns_vertices = ns_offsets[:, np.newaxis] + ns_ccd_corners[np.newaxis, :]


def get_footprint(center):
    """Return the footprint of the ZTF camera centered at the given position.

    Parameters
    ----------
    center : astropy.coordinates.SkyCoord
        The center of the field or fields: either a single (scalar) sky coord
        or an array of sky coordinates of any size.

    Returns
    -------
    coords : astropy.coordinates.SkyCoord
        An array of coordinates. If the center is a single sky coordinate, then
        the returned array has shape (64, 4), representing the vertices of 64
        quadrilaterals. If the center is an array of shape (n, m, ...), then
        the returned array has shape (n, m, ..., 64, 4).
    """
    return SkyCoord(
        ew_vertices, ns_vertices,
        frame=center[..., np.newaxis, np.newaxis].skyoffset_frame()
    ).icrs


center = SkyCoord(0 * u.deg, 0 * u.deg)
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='astro zoom', center='0d 0d', radius=5 * u.deg)
for i, verts in enumerate(get_footprint(center)):
    rc_center = SkyCoord(*verts.cartesian.xyz.sum(1), representation_type='cartesian')
    rc_center.representation_type = 'unitspherical'
    ax.add_patch(plt.Polygon(np.column_stack((verts.ra.deg, verts.dec.deg)), edgecolor='k', facecolor='none', transform=ax.get_transform('world')))
    ax.text(rc_center.ra.deg, rc_center.dec.deg, str(i), transform=ax.get_transform('world'), ha='center', va='center')

### Fetch ZTF field grid

Let's download the ZTF field grid from GitHub.

(Note that the ZTF field IDs start from 1, and that there are some ranges that are skipped, so the index of a row in this array does not exactly match the field ID. This won't matter for this exercise, but just be aware of it if you are reusing the code in this notebook!)

In [ ]:
url = 'https://github.com/ZwickyTransientFacility/ztf_information/raw/master/field_grid/ZTF_Fields.txt'
filename = download_file(url)
field_grid = QTable(np.recfromtxt(filename, comments='%', usecols=range(3), names=['field_id', 'ra', 'dec']))
field_grid['coord'] = SkyCoord(field_grid.columns.pop('ra') * u.deg, field_grid.columns.pop('dec') * u.deg)
field_grid

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.axes(projection='astro mollweide')
for row in field_grid:
    coords = SkyCoord(
        [ew_total, -ew_total, -ew_total, ew_total],
        [ns_total, ns_total, -ns_total, -ns_total],
        frame=row['coord'].skyoffset_frame()
    ).icrs
    ax.add_patch(plt.Polygon(
        np.column_stack((coords.ra.deg, coords.dec.deg)),
        alpha=0.5,
        facecolor='none',
        edgecolor='black',
        transform=ax.get_transform('world')
    ))

### Select fields that are observable

We'll filter for fields that are observable at any point during the night.

In [ ]:
# Observer site
observer = astroplan.Observer.at_site('Palomar')

# Find the earliest possible start time of observations.
# If it's night time (defined here as the sun being at least 18°
# below the horizon for astronomical twilight), then the start
# time is the time of the event. Otherwise, it's the time of sunset.
night_horizon = -18 * u.deg
if observer.is_night(event_time, horizon=night_horizon):
    start_time = event_time
else:
    start_time = observer.sun_set_time(
        event_time, horizon=night_horizon, which='next')

# Find the latest possible end time of observations: the time of sunrise.
end_time = observer.sun_rise_time(
    start_time, horizon=night_horizon, which='next')


min_airmass = 2.5 * u.dimensionless_unscaled
airmass_horizon = (90 * u.deg - np.arccos(1 / min_airmass))
targets = field_grid['coord']

# Find the time that each field rises and sets above an airmass of 2.5.
target_start_time = Time(np.where(
    observer.target_is_up(start_time, targets, horizon=airmass_horizon),
    start_time,
    observer.target_rise_time(start_time, targets, which='next', horizon=airmass_horizon)))
target_start_time.format = 'iso'

# Find the time that each field sets below the airmass limit. If the target
# is always up (i.e., it's circumpolar) or if it sets after surnsise,
# then set the end time to sunrise.
target_end_time = observer.target_set_time(
    target_start_time, targets, which='next', horizon=airmass_horizon)
target_end_time[
    (target_end_time.mask & ~target_start_time.mask) | (target_end_time > end_time)
] = end_time
target_end_time.format = 'iso'

# Select fields that are observable for long enough for at least one exposure
# sequence of 1800 seconds.
exposure_time = 1800 * u.second
field_grid['start_time'] = target_start_time
field_grid['end_time'] = target_end_time
observable_fields = field_grid[target_end_time - target_start_time >= exposure_time]
observable_fields

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.axes(projection='astro mollweide')
for row in observable_fields:
    coords = SkyCoord(
        [ew_total, -ew_total, -ew_total, ew_total],
        [ns_total, ns_total, -ns_total, -ns_total],
        frame=row['coord'].skyoffset_frame()
    ).icrs
    ax.add_patch(plt.Polygon(
        np.column_stack((coords.ra.deg, coords.dec.deg)),
        alpha=0.5,
        facecolor='none',
        edgecolor='black',
        transform=ax.get_transform('world')
    ))

### HEALPix representation of ZTF focal plane footprint

Now we will model the ZTF focal plane by computing the HEALPix pixel indices contained within the region.

Play the value of `nside` to adjust the HEALPix resolution. It must be an integer power of 2.

**Exercise 7.** What is the minimum HEALPix resolution to resolve all of the chip gaps?

Note that values much larger than `nside=128` will result in problems that take a very long time to solve with HiGHS, and for which you might need to use a professional solver like [CPLEX](https://www.ibm.com/products/ilog-cplex-optimization-studio/cplex-optimizer), [Gurobi](https://www.gurobi.com), or [FICO Xpress](https://www.fico.com/en/products/fico-xpress-optimization).

In [ ]:
# fill me in: play with the value of nside!
hpx = HEALPix(nside=128, frame=ICRS())

footprint = np.moveaxis(
    get_footprint(SkyCoord(0 * u.deg, 0 * u.deg)).cartesian.xyz.value, 0, -1)
footprint_healpix = np.unique(np.concatenate(
    [hp.query_polygon(hpx.nside, v, nest=(hpx.order == 'nested')) for v in footprint]))

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='astro zoom', center='0d 0d', radius=5 * u.deg)
for boundary in hpx.boundaries_skycoord(footprint_healpix, 1):
    ax.add_patch(plt.Polygon(np.column_stack((boundary.ra.deg, boundary.dec.deg)), edgecolor='k', facecolor='lightgray', transform=ax.get_transform('world')))


Now that we have selected a nice resolution, compute the footprints of every ZTF field as HEALPix indices. Also downsample the sky map to the same resolution.

In [ ]:
footprints = np.moveaxis(get_footprint(observable_fields['coord']).cartesian.xyz.value, 0, -1)
footprints_healpix = [
    np.unique(np.concatenate([hp.query_polygon(hpx.nside, v) for v in footprint]))
    for footprint in tqdm(footprints)]

prob = hp.ud_grade(skymap, hpx.nside, power=-2)

Now it's your turn to do the math!

### Optimal tiling using max weighted coverage

**Exercise 8.** Which 30 fields should ZTF observe in order to follow up this GW event? How much probability is contained in those fields?

Hint: Construct a max weighted coverage model of this problem. Express it as a MILP. Write the MILP model in HiGHS and solve it.

In [ ]:
model = highspy.Highs()

field_vars = [model.addBinary() for _ in range(len(footprints))]
pixel_vars = [model.addBinary() for _ in range(hpx.npix)]

# fill me in...
model.addConstr(...)
model.maximize(...)

How much probability is contained in the fields that we selected?

In [ ]:
model.getObjectiveValue()

Which fields did you pick to observe?

In [ ]:
selected_fields = observable_fields[np.asarray(model.variableValues(field_vars), dtype=bool)]
selected_fields

Plot the fields that you selected to observe.

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.axes(projection='astro mollweide')
for row in selected_fields:
    coords = SkyCoord(
        [ew_total, -ew_total, -ew_total, ew_total],
        [ns_total, ns_total, -ns_total, -ns_total],
        frame=row['coord'].skyoffset_frame()
    ).icrs
    ax.add_patch(plt.Polygon(
        np.column_stack((coords.ra.deg, coords.dec.deg)),
        alpha=0.5,
        facecolor='lightgray',
        edgecolor='black',
        transform=ax.get_transform('world')
    ))
ax.grid()

### Optimal scheduling

Now that we have identified which fields to observe, it's time to schedule them: we must decide which fields to observe in what order. For this problem we are going to make use of our **Logical constraints ↔︎ ILP translation dictionary** from the lecture.

We start our model by creating two vectors of decision variables:
- The binary vector $\mathbf{x} = (x_1, x_2, \dots, x_n)$, where $x_i$ is 1 if we decide to observe field $i$, otherwise 0.
- The real vector $\mathbf{t} = (t_1, t_2, \dots, t_n)$, where $t_i$ is the start time of the observation of field $i$. Each of these variables will have a lower bound that is the earliest time that the field can be observed, and an upper bound that is the latest time that it can be observed.

We want to maximize the total number of fields that we can observe, $\sum_i x_i$.

We need to add some constraints to ensure that the observations do not overlap: we can observe at most one field at a time. If the duration of each exposure is $\delta$, then in "pseudocode" these constraints are:

$$
\text{if } x_i = 1, \text{ then } t_i + \delta \leq t_j \text{ or } t_j + \delta \leq t_i \text{ for all } j < i.
$$

**Exercise 9**. Using the **Logical constraints ↔︎ ILP translation dictionary**, express the no-overlap constraints above as a system of linear inequalities suitable for a MILP.

Hint: you'll probably use the big-M trick!

**Exercise 10**. Complete the code below to model the schedule as an MILP and find the order in which to obseve the ZTF fields.

In [ ]:
model = highspy.Highs()

# Limit run time; this can take while to solve!
# The solver will try to find the best feasible solution within the time limit.
model.setOptionValue('time_limit', 60)

delta = exposure_time.to_value(u.day)
# Fill me in
M = ...

t = [model.addVariable(
        lb=(row['start_time'] - start_time).to_value(u.day),
        ub=(row['end_time'] - start_time - exposure_time).to_value(u.day),
     ) for row in selected_fields]
x = [model.addBinary() for _ in range(len(t))]
s = [[model.addBinary() for j in range(i)] for i in range(len(t))]

# Fill me in, add constraints...
model.addConstr(...)
model.maximize(...)

Which fields did you choose to observe?

In [ ]:
scheduled_fields = QTable(selected_fields)
scheduled_fields['scheduled_start_time'] = Time(model.variableValues(t) + start_time, format='mjd')
scheduled_fields['scheduled_start_time'].format = 'iso'
scheduled_fields['scheduled_end_time'] = scheduled_fields['scheduled_start_time'] + exposure_time
scheduled_fields = scheduled_fields[np.asarray(model.variableValues(x), dtype=bool)]
scheduled_fields.sort('scheduled_start_time')
scheduled_fields

Plot the scheduled intervals to check that they do not overlap.

In [ ]:
ax = plt.axes()
ax.hlines(
    np.arange(len(scheduled_fields)),
    scheduled_fields['scheduled_start_time'].mjd,
    scheduled_fields['scheduled_end_time'].mjd)
ax.set_yticks(np.arange(len(scheduled_fields)), scheduled_fields['field_id'].astype(str))
ax.set_xlabel('Observation time (MJD)')
ax.set_ylabel('field ID')